In [182]:
# import modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

# import data from sklearn

data = load_breast_cancer()
#print(data)

# make a dataframe from the data
df = pd.DataFrame(data.data, columns = data.feature_names)

print(df.head())

# make X and y as predictor and outcome variables to the model

X = df[df.columns.to_list()]
y = data.target

# standardize the X values by initalizing the StandardScaler then fit and transform the data
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# split the data into train and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

# make a LogisticRegression model, fit the training X and y values and then predict y values with using test x values

lrm = LogisticRegression()
model = lrm.fit(X_train, y_train)
THRESHOLD = 0.30
y_pred = np.where(model.predict_proba(X_test)[:,1] > THRESHOLD, 1, 0)

print(y_test)
print(y_pred)

# Evaluate the model by using from sklearn.metrics import confusion_matrix and accuracy_score

print(f'The confusion matrix is below:\n{confusion_matrix(y_test, y_pred)}\nThis represents the true negatives, the false positives in the first column then the false negatives and true positives successively.\n')
print(f'The model is {round(100*accuracy_score(y_test, y_pred))}% accurate')


   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst radius  worst texture  worst perimeter  \
0           